## Model Building and Training

### Data Preparation

In [1]:
import pandas as pd
import numpy as np
import mlflow
import mlflow.sklearn  # For sklearn models
import mlflow.keras    # For Keras models

In [2]:
fraud_df = pd.read_csv("../Notebooks/fraud_dataset.csv")
creditcard_df = pd.read_csv("../data/creditcard.csv")

In [3]:
fraud_df.head(5)

,user_id,signup_time,purchase_time,purchase_value,device_id,age,ip_address,class,country,transaction_count,transaction_velocity,hour_of_day,day_of_week,browser_FireFox,browser_IE,browser_Opera,browser_Safari,source_Direct,source_SEO,sex_M
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,-0.160204,QVPSPJUOCKZAR,39,7.327584e+08,0,Japan,0.0,0.0,2,5,False,False,False,False,False,True,True
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,-1.142592,EOGFQPIZPYXFZ,53,3.503114e+08,0,United States,0.0,0.0,1,0,False,False,False,False,False,False,False
2,1359,2015-01-01 18:52:44,2015-01-01 18:52:45,-1.197169,YSSKYOSJHPPLJ,53,2.621474e+09,1,Unknown,0.0,0.0,18,3,False,False,True,False,False,True,True
3,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,0.385567,ATGTXKYKUDUQN,41,3.840542e+09,0,Unknown,0.0,0.0,13,0,False,False,False,True,False,True,True
4,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,0.112681,NAUITBZFJKHWW,45,4.155831e+08,0,United States,0.0,0.0,18,2,False,False,False,True,False,False,True


In [4]:
creditcard_df.head(5)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [5]:
fraud_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151112 entries, 0 to 151111
Data columns (total 20 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   user_id               151112 non-null  int64  
 1   signup_time           151112 non-null  object 
 2   purchase_time         151112 non-null  object 
 3   purchase_value        151112 non-null  float64
 4   device_id             151112 non-null  object 
 5   age                   151112 non-null  int64  
 6   ip_address            151112 non-null  float64
 7   class                 151112 non-null  int64  
 8   country               151112 non-null  object 
 9   transaction_count     151112 non-null  float64
 10  transaction_velocity  151112 non-null  float64
 11  hour_of_day           151112 non-null  int64  
 12  day_of_week           151112 non-null  int64  
 13  browser_FireFox       151112 non-null  bool   
 14  browser_IE            151112 non-null  bool   
 15  

#### Feature and Target Separation

In [6]:
# Correcting data types
fraud_df['signup_time'] = pd.to_datetime(fraud_df['signup_time'])
fraud_df['purchase_time'] = pd.to_datetime(fraud_df['purchase_time'])
fraud_df['ip_address'] = fraud_df['ip_address'].fillna(0).astype(int)

In [7]:
# Create a new feature: time difference between signup and purchase (in minutes)
fraud_df['time_diff_minutes'] = (fraud_df['purchase_time'] - fraud_df['signup_time']).dt.total_seconds() / 60.0

# Extract additional features from 'purchase_time'
fraud_df['purchase_hour'] = fraud_df['purchase_time'].dt.hour
fraud_df['purchase_day'] = fraud_df['purchase_time'].dt.day
fraud_df['purchase_month'] = fraud_df['purchase_time'].dt.month

In [8]:
# Drop the original 'signup_time' and 'purchase_time' columns
fraud_df = fraud_df.drop(columns=['signup_time','purchase_time'])

In [9]:
from sklearn.preprocessing import LabelEncoder

#  Label encoding for categorical columns in Fraud_Data
label_encoder = LabelEncoder()
fraud_df['device_id'] = label_encoder.fit_transform(fraud_df['device_id'])

In [16]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# Replace 'Unknown' with NaN
fraud_df = fraud_df.replace('Unknown', np.nan)

categorical_cols = ['country']

# Preprocessing for categorical data
categorical_transformer = SimpleImputer(strategy='most_frequent') 

# Preprocessing for categorical data (e.g., filling missing values with most frequent value and one-hot encoding)
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Fill missing values in categorical columns
    ('onehot', OneHotEncoder(handle_unknown='ignore'))  # One-hot encode the categorical column
])

# Combine preprocessing steps for numerical and categorical columns
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_cols)
    ])

### Preprocessing

In [10]:
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# Replace 'Unknown' with NaN
fraud_df = fraud_df.replace('Unknown', np.nan)

# Define categorical and numerical columns
categorical_cols = ['country']  # Add other categorical columns if needed
numerical_cols = fraud_df.select_dtypes(include=['int64', 'float64', 'bool']).columns.tolist()

# Preprocessing for numerical data (e.g., filling missing values with mean)
numerical_transformer = SimpleImputer(strategy='mean')

# Preprocessing for categorical data (e.g., filling missing values with most frequent value and one-hot encoding)
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Fill missing values in categorical columns
    ('onehot', OneHotEncoder(handle_unknown='ignore'))  # One-hot encode the categorical column
])

# Combine preprocessing steps for numerical and categorical columns
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

In [17]:
# For Fraud_Data.csv
y_fraud = fraud_df['class']
X_fraud = fraud_df.drop(columns=['class'])


# For creditcard.csv
y_creditcard = creditcard_df['Class']
X_creditcard = creditcard_df.drop(columns=['Class'])


#### Train-Test Split:

In [12]:
import mlflow
import mlflow.sklearn  # For sklearn models
from sklearn.model_selection import train_test_split

# Set up an experiment
mlflow.set_experiment("Fraud Detection Experiment")

# Split Fraud Data
X_fraud_train, X_fraud_test, y_fraud_train, y_fraud_test = train_test_split(X_fraud, y_fraud, test_size=0.2, random_state=42)

# Split Credit Card Data
X_creditcard_train, X_creditcard_test, y_creditcard_train, y_creditcard_test = train_test_split(X_creditcard, y_creditcard, test_size=0.2, random_state=42)

2024/10/21 20:41:44 INFO mlflow.tracking.fluent: Experiment with name 'Fraud Detection Experiment' does not exist. Creating a new experiment.


In [13]:
X_fraud_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 120889 entries, 119187 to 121958
Data columns (total 21 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   user_id               120889 non-null  int64  
 1   purchase_value        120889 non-null  float64
 2   device_id             120889 non-null  int32  
 3   age                   120889 non-null  int64  
 4   ip_address            120889 non-null  int32  
 5   country               58496 non-null   object 
 6   transaction_count     120889 non-null  float64
 7   transaction_velocity  120889 non-null  float64
 8   hour_of_day           120889 non-null  int64  
 9   day_of_week           120889 non-null  int64  
 10  browser_FireFox       120889 non-null  bool   
 11  browser_IE            120889 non-null  bool   
 12  browser_Opera         120889 non-null  bool   
 13  browser_Safari        120889 non-null  bool   
 14  source_Direct         120889 non-null  bool   
 15  

### Model Selection

#### Traditional Models:

#### Extract useful features from the datetime columns

In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import classification_report, accuracy_score


# Initialize models
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting': GradientBoostingClassifier()
}


    
   

# Train and log results for sklearn models (Fraud Data)
for name, model in models.items():
    with mlflow.start_run(run_name=name):
        # Create a pipeline with preprocessing and model
        clf = Pipeline(steps=[('preprocessor', preprocessor),
                          ('classifier', model)])
        clf.fit(X_fraud_train, y_fraud_train)
        y_pred_fraud = clf.predict(X_fraud_test)
        
        # Log parameters and metrics
        mlflow.log_params(model.get_params())
        accuracy = accuracy_score(y_fraud_test, y_pred_fraud)
        mlflow.log_metric("accuracy", accuracy)
        mlflow.sklearn.log_model(model, name)

        # Output classification report
        print(f"{name} - Fraud Data:")
        print(classification_report(y_fraud_test, y_pred_fraud))
        print("Accuracy:", accuracy)
        
        mlflow.end_run()  # End the current run

2024/10/21 20:43:55 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Logistic Regression - Fraud Data:
              precision    recall  f1-score   support

           0       0.91      1.00      0.95     27373
           1       0.00      0.00      0.00      2850

    accuracy                           0.91     30223
   macro avg       0.45      0.50      0.48     30223
weighted avg       0.82      0.91      0.86     30223

Accuracy: 0.9057009562253913


c:\Users\lenovo\Documents\10acadmy\FraudDetection-for-E-commerce-Banking\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\lenovo\Documents\10acadmy\FraudDetection-for-E-commerce-Banking\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\lenovo\Documents\10acadmy\FraudDetection-for-E-commerce-Banking\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use 

Decision Tree - Fraud Data:
              precision    recall  f1-score   support

           0       0.91      1.00      0.95     27373
           1       0.44      0.00      0.00      2850

    accuracy                           0.91     30223
   macro avg       0.68      0.50      0.48     30223
weighted avg       0.86      0.91      0.86     30223

Accuracy: 0.9056678688416107


2024/10/21 20:44:21 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Random Forest - Fraud Data:
              precision    recall  f1-score   support

           0       0.91      1.00      0.95     27373
           1       0.44      0.00      0.00      2850

    accuracy                           0.91     30223
   macro avg       0.68      0.50      0.48     30223
weighted avg       0.86      0.91      0.86     30223

Accuracy: 0.9056678688416107


2024/10/21 20:44:38 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Gradient Boosting - Fraud Data:
              precision    recall  f1-score   support

           0       0.91      1.00      0.95     27373
           1       0.00      0.00      0.00      2850

    accuracy                           0.91     30223
   macro avg       0.45      0.50      0.48     30223
weighted avg       0.82      0.91      0.86     30223

Accuracy: 0.9057009562253913


c:\Users\lenovo\Documents\10acadmy\FraudDetection-for-E-commerce-Banking\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\lenovo\Documents\10acadmy\FraudDetection-for-E-commerce-Banking\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\lenovo\Documents\10acadmy\FraudDetection-for-E-commerce-Banking\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use 

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, LSTM, Embedding, Dropout
from sklearn.preprocessing import StandardScaler

# # Scale data for Neural Networks
# scaler = StandardScaler()
# X_creditcard_scaled = scaler.fit_transform(X_creditcard)
# Scaling data for neural networks
scaler = StandardScaler()
X_creditcard_train_scaled = scaler.fit_transform(X_creditcard_train)
X_creditcard_test_scaled = scaler.transform(X_creditcard_test)

# # Reshape for CNN (assuming 2D structure, modify according to your features)
# X_creditcard_cnn = X_creditcard_scaled.reshape(-1, X_creditcard_scaled.shape[1], 1)

# MLP Model
def create_mlp_model(input_shape):
    model = Sequential()
    model.add(Dense(64, activation='relu', input_shape=(input_shape,)))
    model.add(Dropout(0.5))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    return model

mlp_model = create_mlp_model(X_creditcard.shape[1])
mlp_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# Start a new MLflow run for MLP
with mlflow.start_run(run_name="MLP"):
    mlflow.log_param("model_type", "MLP")
    mlflow.log_param("epochs", 10)
    mlp_model.fit(X_creditcard_train_scaled, y_creditcard_train, epochs=10, batch_size=32, validation_data=(X_creditcard_test_scaled, y_creditcard_test))

    # Log model and accuracy
    accuracy = mlp_model.evaluate(X_creditcard_test_scaled, y_creditcard_test)[1]
    mlflow.log_metric("accuracy", accuracy)
    mlflow.keras.log_model(mlp_model, "mlp_model")
    mlflow.end_run()  # End the current run
# mlp_model.fit(X_creditcard_train, y_creditcard_train, epochs=10, batch_size=32, validation_data=(X_creditcard_test, y_creditcard_test))


c:\Users\lenovo\Documents\10acadmy\FraudDetection-for-E-commerce-Banking\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
7121/7121 ━━━━━━━━━━━━━━━━━━━━ 37s 5ms/step - accuracy: 0.9957 - loss: 0.0254 - val_accuracy: 0.9993 - val_loss: 0.0036
Epoch 2/10
7121/7121 ━━━━━━━━━━━━━━━━━━━━ 43s 6ms/step - accuracy: 0.9993 - loss: 0.0043 - val_accuracy: 0.9994 - val_loss: 0.0033
Epoch 3/10
7121/7121 ━━━━━━━━━━━━━━━━━━━━ 78s 5ms/step - accuracy: 0.9993 - loss: 0.0041 - val_accuracy: 0.9994 - val_loss: 0.0030
Epoch 4/10
7121/7121 ━━━━━━━━━━━━━━━━━━━━ 39s 5ms/step - accuracy: 0.9994 - loss: 0.0033 - val_accuracy: 0.9993 - val_loss: 0.0033
Epoch 5/10
7121/7121 ━━━━━━━━━━━━━━━━━━━━ 47s 6ms/step - accuracy: 0.9995 - loss: 0.0029 - val_accuracy: 0.9994 - val_loss: 0.0031
Epoch 6/10
7121/7121 ━━━━━━━━━━━━━━━━━━━━ 78s 6ms/step - accuracy: 0.9995 - loss: 0.0029 - val_accuracy: 0.9993 - val_loss: 0.0032
Epoch 7/10
7121/7121 ━━━━━━━━━━━━━━━━━━━━ 48s 7ms/step - accuracy: 0.9995 - loss: 0.0027 - val_accuracy: 0.9994 - val_loss: 0.0032
Epoch 8/10
7121/7121 ━━━━━━━━━━━━━━━━━━━━ 77s 6ms/step - accuracy: 0.9994 - loss: 0

2024/10/21 20:54:30 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2024/10/21 20:54:58 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense

# Scale data for Neural Networks
scaler = StandardScaler()
X_creditcard_scaled = scaler.fit_transform(X_creditcard)
X_creditcard_train = scaler.fit_transform(X_creditcard_train)
X_creditcard_test = scaler.fit_transform(X_creditcard_test)

# X_creditcard_cnn = X_creditcard_train.reshape(-1, X_creditcard_scaled.shape[1], 1)
# X_creditcard_cnn_test = X_creditcard_test.reshape(-1, X_creditcard_scaled.shape[1], 1)
# Reshape data for CNN
X_creditcard_train_cnn = X_creditcard_train_scaled.reshape(-1, X_creditcard_train_scaled.shape[1], 1)
X_creditcard_test_cnn = X_creditcard_test_scaled.reshape(-1, X_creditcard_test_scaled.shape[1], 1)


# Adjust the CNN model for 1D data
cnn_model = Sequential()

# Assuming X_creditcard_cnn has shape (samples, 30 features, 1 channel)
cnn_model.add(Conv1D(32, kernel_size=3, activation='relu', input_shape=(30, 1)))
cnn_model.add(Flatten())
cnn_model.add(Dense(1, activation='sigmoid'))

cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# Start a new MLflow run for CNN
with mlflow.start_run(run_name="CNN"):
    cnn_model.fit(X_creditcard_train_cnn, y_creditcard_train, epochs=10, batch_size=32)
    
    # Log model and accuracy
    accuracy = cnn_model.evaluate(X_creditcard_test_cnn, y_creditcard_test)[1]
    mlflow.log_metric("accuracy", accuracy)
    mlflow.keras.log_model(cnn_model, "cnn_model")
    mlflow.end_run()
# Train the model
# cnn_model.fit(X_creditcard_cnn, y_creditcard_train, epochs=10, batch_size=32)


c:\Users\lenovo\Documents\10acadmy\FraudDetection-for-E-commerce-Banking\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
7121/7121 ━━━━━━━━━━━━━━━━━━━━ 27s 3ms/step - accuracy: 0.9965 - loss: 0.0217
Epoch 2/10
7121/7121 ━━━━━━━━━━━━━━━━━━━━ 25s 4ms/step - accuracy: 0.9993 - loss: 0.0032
Epoch 3/10
7121/7121 ━━━━━━━━━━━━━━━━━━━━ 23s 3ms/step - accuracy: 0.9993 - loss: 0.0033
Epoch 4/10
7121/7121 ━━━━━━━━━━━━━━━━━━━━ 25s 3ms/step - accuracy: 0.9993 - loss: 0.0033
Epoch 5/10
7121/7121 ━━━━━━━━━━━━━━━━━━━━ 25s 4ms/step - accuracy: 0.9995 - loss: 0.0025
Epoch 6/10
7121/7121 ━━━━━━━━━━━━━━━━━━━━ 42s 4ms/step - accuracy: 0.9994 - loss: 0.0032
Epoch 7/10
7121/7121 ━━━━━━━━━━━━━━━━━━━━ 25s 4ms/step - accuracy: 0.9995 - loss: 0.0026
Epoch 8/10
7121/7121 ━━━━━━━━━━━━━━━━━━━━ 25s 3ms/step - accuracy: 0.9995 - loss: 0.0024
Epoch 9/10
7121/7121 ━━━━━━━━━━━━━━━━━━━━ 27s 4ms/step - accuracy: 0.9995 - loss: 0.0024
Epoch 10/10
7121/7121 ━━━━━━━━━━━━━━━━━━━━ 40s 4ms/step - accuracy: 0.9995 - loss: 0.0025
1781/1781 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9994 - loss: 0.0033


2024/10/21 21:04:23 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2024/10/21 21:04:35 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [21]:
# LSTM Model (assuming sequential data)
lstm_model = Sequential()
lstm_model.add(LSTM(50, activation='relu', input_shape=(X_creditcard.shape[1], 1)))
lstm_model.add(Dense(1, activation='sigmoid'))
lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# Start a new MLflow run for LSTM
with mlflow.start_run(run_name="LSTM"):
    lstm_model.fit(X_creditcard_train_cnn, y_creditcard_train, epochs=10, batch_size=32)
    
    # Log model and accuracy
    accuracy = lstm_model.evaluate(X_creditcard_test_cnn, y_creditcard_test)[1]
    mlflow.log_metric("accuracy", accuracy)
    mlflow.keras.log_model(lstm_model, "lstm_model")
    mlflow.end_run()
# lstm_model.fit(X_creditcard_train, y_creditcard_train, epochs=10, batch_size=32)

c:\Users\lenovo\Documents\10acadmy\FraudDetection-for-E-commerce-Banking\.venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
7121/7121 ━━━━━━━━━━━━━━━━━━━━ 111s 15ms/step - accuracy: 0.9959 - loss: 103.4355
Epoch 2/10
7121/7121 ━━━━━━━━━━━━━━━━━━━━ 146s 15ms/step - accuracy: 0.9988 - loss: 0.0054
Epoch 3/10
7121/7121 ━━━━━━━━━━━━━━━━━━━━ 108s 15ms/step - accuracy: 0.9992 - loss: 0.0042
Epoch 4/10
7121/7121 ━━━━━━━━━━━━━━━━━━━━ 140s 15ms/step - accuracy: 0.9991 - loss: 0.0048
Epoch 5/10
7121/7121 ━━━━━━━━━━━━━━━━━━━━ 112s 16ms/step - accuracy: 0.9991 - loss: 0.0079
Epoch 6/10
7121/7121 ━━━━━━━━━━━━━━━━━━━━ 138s 15ms/step - accuracy: 0.9994 - loss: 0.0057
Epoch 7/10
7121/7121 ━━━━━━━━━━━━━━━━━━━━ 141s 15ms/step - accuracy: 0.9992 - loss: 0.0066
Epoch 8/10
7121/7121 ━━━━━━━━━━━━━━━━━━━━ 140s 15ms/step - accuracy: 0.9991 - loss: 0.0682
Epoch 9/10
7121/7121 ━━━━━━━━━━━━━━━━━━━━ 111s 16ms/step - accuracy: 0.9991 - loss: 0.0061
Epoch 10/10
7121/7121 ━━━━━━━━━━━━━━━━━━━━ 138s 15ms/step - accuracy: 0.9991 - loss: 14.1526
1781/1781 ━━━━━━━━━━━━━━━━━━━━ 13s 7ms/step - accuracy: 0.9990 - loss: 0.0070


2024/10/21 21:27:33 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2024/10/21 21:27:59 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [38]:
# Function to evaluate model performance
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    y_pred_classes = (y_pred > 0.5).astype(int)  # Thresholding for binary classification
    print(classification_report(y_test, y_pred_classes))
    print("Accuracy:", accuracy_score(y_test, y_pred_classes))

# Evaluate the MLP model
evaluate_model(mlp_model, X_creditcard_test, y_creditcard_test)

# Evaluate the CNN model
evaluate_model(cnn_model, X_creditcard_test, y_creditcard_test)

# Evaluate the LSTM model
evaluate_model(lstm_model, X_creditcard_test, y_creditcard_test)


1781/1781 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step


c:\Users\lenovo\Documents\10acadmy\FraudDetection-for-E-commerce-Banking\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\lenovo\Documents\10acadmy\FraudDetection-for-E-commerce-Banking\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\lenovo\Documents\10acadmy\FraudDetection-for-E-commerce-Banking\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use 

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.00      0.00      0.00        98

    accuracy                           1.00     56962
   macro avg       0.50      0.50      0.50     56962
weighted avg       1.00      1.00      1.00     56962

Accuracy: 0.9982795547909132
1781/1781 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.97      0.66      0.79        98

    accuracy                           1.00     56962
   macro avg       0.98      0.83      0.89     56962
weighted avg       1.00      1.00      1.00     56962

Accuracy: 0.999385555282469
1781/1781 ━━━━━━━━━━━━━━━━━━━━ 13s 7ms/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.80      0.72      0.76        98

    accuracy                          

In [22]:
import shap

# Create SHAP explainer for the model
explainer = shap.KernelExplainer(cnn_model.predict, X_creditcard_train)

# Get SHAP values for test data
shap_values = explainer.shap_values(X_creditcard_test)

# Plot summary of feature importance
shap.summary_plot(shap_values, X_creditcard_test)

c:\Users\lenovo\Documents\10acadmy\FraudDetection-for-E-commerce-Banking\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


7121/7121 ━━━━━━━━━━━━━━━━━━━━ 18s 2ms/step


Using 227845 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.
  0%|          | 0/56962 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 178ms/step


  0%|          | 0/56962 [06:06<?, ?it/s]


MemoryError: Unable to allocate 107. GiB for an array with shape (2108, 6835350) and data type float64

In [25]:
import joblib
from datetime import datetime

# Serialize the model with a timestamp
timestamp = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
cnn_model.save(f"cnn_model_{timestamp}.h5")
joblib.dump(cnn_model, f"cnn_model_{timestamp}.pkl")
joblib.dump(mlp_model, f"mlp_model_{timestamp}.pkl")
joblib.dump(RandomForestClassifier(), f"RandomForestClassifier_{timestamp}.pkl")

['RandomForestClassifier_2024-10-22-12-34-32.pkl']

In [26]:
# Reduce background data to 100 samples using shap.sample
background_data = shap.sample(X_creditcard_train, 100)

In [27]:
# Create SHAP explainer with reduced background data
explainer = shap.KernelExplainer(cnn_model.predict, background_data)

# Compute SHAP values for the test data
shap_values = explainer.shap_values(X_creditcard_test, nsamples=100)  # You can limit the number of samples to avoid memory issues

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step


  0%|          | 0/56962 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step


  0%|          | 1/56962 [00:04<64:05:54,  4.05s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


  1%|          | 351/56962 [10:34<34:14:29,  2.18s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
 66/313 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

  1%|          | 351/56962 [10:34<28:26:42,  1.81s/it]


KeyboardInterrupt: 

In [3]:
from keras.models import load_model

# Load the model in .h5 format
cnn_model = load_model('cnn_model_2024-10-22-12-34-32.h5')

In [4]:
# Plot the SHAP summary plot
shap.summary_plot(shap_values, X_creditcard_test)

NameError: name 'shap_values' is not defined

In [5]:
import shap

# Use DeepExplainer for neural network models (like CNN)
explainer = shap.DeepExplainer(cnn_model, X_creditcard_train[:100])  # Use a smaller training subset as background

# Compute SHAP values for a smaller test subset
shap_values = explainer.shap_values(X_creditcard_test[:100])

# Plot the summary plot
shap.summary_plot(shap_values, X_creditcard_test[:100])

NameError: name 'X_creditcard_train' is not defined